In [1]:
from keras_tuner import HyperModel
import keras_tuner as kt

import numpy as np
import pandas as pd
#!pip install keras_tuner
#import keras_tuner

import keras
from keras import models, layers, metrics, Sequential
from keras.layers import Dense, BatchNormalization, Dropout
import tensorflow as tf
from keras.regularizers import l1,l2,l1_l2
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Input, Dense, Multiply, Concatenate
from tensorflow.keras.models import Model

import tensorflow as tf
import random

import keras_tuner
from keras_tuner import HyperModel
import keras_tuner as kt

from sklearn.preprocessing import MinMaxScaler

In [2]:
# Import Dataset
import pandas as pd
path = 'dataset/'
firm_df = pd.read_pickle(path+'firm_df49.pkl')
firm_df.index = pd.to_datetime(firm_df.index).to_period('M')

# macro-economic variables
# There is one features which has 37 missing values but before year 1980
macro_df = pd.read_pickle(path+'macro_df.pkl')
macro_df.index = pd.to_datetime(macro_df.index, format='%m/%d/%Y')
macro_df.rename(columns={"ep": "ep_macro"}, inplace=True)
macro_index = macro_df.index[1:]
macro_new = macro_df.iloc[:-1, :]
macro_new.index = macro_index
macro_new.index = macro_new.index.to_period('M')

# Scale macro-features
from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler(feature_range=(-0.5, 0.5))
#macro_scaled = scaler.fit_transform(macro_new)
#macro_scaled_df = pd.DataFrame(macro_scaled, columns=macro_new.columns)
#macro_scaled_df.index = macro_new.index

# merged data 1
merged_df = pd.merge(firm_df, macro_new, left_on='jdate', right_on='sasdate', how='inner')
merged_df.index = firm_df.index

In [3]:
char_core = ['acc','agr','beta','bm','cash','cashpr','cfp','chatoia','chcsho','chfeps','chinv',
       'chmom','chpmia','chtx','currat','depr','dy','ear','ep','gma','grcapx','grltnoa',
       'ill','indmom','invest','lev','lgr','maxret','mom12m','mom1m','mom36m','mve','nincr',
       'orgcap','pchgm_pchsale','pchsale_pchinvt','pchsale_pchrect','pchsale_pchxsga',
       'retvol','roaq','roavol','roeq','salecash','saleinv','sgr','sp','std_dolvol','std_turn','turn']

info_list = ['fyear','year','jyear','permno','ticker','comnam','exchcd','exchname','siccd',
       'indname','size_class','mve_m','rf','ret','ret_adj','ret_ex','ret_adj_ex',]

macro_col = ['RPI', 'W875RX1', 'DPCERA3M086SBEA', 'CMRMTSPLx', 'RETAILx','INDPRO', 'IPFPNSS', 'IPFINAL', 'IPCONGD', 'IPDCONGD', 'IPNCONGD',
       'IPBUSEQ', 'IPMAT', 'IPDMAT', 'IPNMAT', 'IPMANSICS', 'IPB51222S','IPFUELS', 'CUMFNS', 'HWI', 'HWIURATIO', 'CLF16OV', 'CE16OV',
       'UNRATE', 'UEMPMEAN', 'UEMPLT5', 'UEMP5TO14', 'UEMP15OV','UEMP15T26', 'UEMP27OV', 'CLAIMSx', 'PAYEMS', 'USGOOD',
       'CES1021000001', 'USCONS', 'MANEMP', 'DMANEMP', 'NDMANEMP','SRVPRD', 'USTPU', 'USWTRADE', 'USTRADE', 'USFIRE', 'USGOVT',
       'CES0600000007', 'AWOTMAN', 'AWHMAN', 'HOUST', 'HOUSTNE','HOUSTMW', 'HOUSTS', 'HOUSTW', 'PERMIT', 'PERMITNE', 'PERMITMW',
       'PERMITS', 'PERMITW', 'AMDMNOx', 'ANDENOx', 'AMDMUOx', 'BUSINVx','ISRATIOx', 'M1SL', 'M2SL', 'M2REAL', 'BOGMBASE', 'TOTRESNS',
       'NONBORRES', 'BUSLOANS', 'REALLN', 'NONREVSL', 'CONSPI', 'S&P 500','S&P: indust', 'S&P div yield', 'S&P PE ratio', 'FEDFUNDS',
       'CP3Mx', 'TB3MS', 'TB6MS', 'GS1', 'GS5', 'GS10', 'AAA', 'BAA','COMPAPFFx', 'TB3SMFFM', 'TB6SMFFM', 'T1YFFM', 'T5YFFM', 'T10YFFM',
       'AAAFFM', 'BAAFFM', 'TWEXAFEGSMTHx', 'EXSZUSx', 'EXJPUSx','EXUSUKx', 'EXCAUSx', 'WPSFD49207', 'WPSFD49502', 'WPSID61',
       'WPSID62', 'OILPRICEx', 'PPICMM', 'CPIAUCSL', 'CPIAPPSL','CPITRNSL', 'CPIMEDSL', 'CUSR0000SAC', 'CUSR0000SAD',
       'CUSR0000SAS', 'CPIULFSL', 'CUSR0000SA0L2', 'CUSR0000SA0L5','PCEPI', 'DDURRG3M086SBEA', 'DNDGRG3M086SBEA', 'DSERRG3M086SBEA',
       'CES0600000008', 'CES2000000008', 'CES3000000008', 'UMCSENTx','DTCOLNVHFNM', 'DTCTHFNM', 'INVEST', 'VIXCLSx', 'dp', 'ep_macro', 'b/m',
       'ntis', 'tbl', 'tms', 'dfy', 'svar']

macro_core =  ['dp', 'ep_macro', 'b/m','ntis', 'tbl', 'tms', 'dfy', 'svar']

def datasplit(df, dataset):
    df_train = df[df.index < pd.Period((str(1995)+"-1"),freq='M')]
    df_val = df[(df.index >= pd.Period((str(1995)+"-1"),freq='M')) & (df.index < pd.Period((str(2000)+"-1"),freq='M'))]
    df_test = df[df.index >= pd.Period((str(2000)+"-1"),freq='M')]
    if dataset == "firm":
        X_train = df_train[char_core]
        X_val = df_val[char_core]
        X_test = df_test[char_core]
    elif dataset == 'macro':
        X_train = df_train[macro_col]
        X_val = df_val[macro_col]
        X_test = df_test[macro_col]
    elif dataset == "firm_macro":
        X_train = df_train[char_core+macro_col]
        X_val = df_val[char_core+macro_col]
        X_test = df_test[char_core+macro_col]
    elif dataset == "firm_macro8":
        X_train = df_train[char_core+macro_core]
        X_val = df_val[char_core+macro_core]
        X_test = df_test[char_core+macro_core] 
    y_train = df_train['predicted_return']
    y_val = df_val['predicted_return']
    y_test = df_test['predicted_return']
    return (X_train, y_train, X_val, y_val, X_test, y_test, )

In [4]:
def my_metric_fn(y_true, y_pred):
    num = tf.reduce_mean(tf.square(y_true - y_pred))
    den = tf.reduce_mean(tf.square(y_true))
    return 1 - num / den

class MyHyperModel(HyperModel,):
    def __init__(self, input_shape,random_seed=None,):
        self.input_value = input_shape
        if random_seed is not None:
            random.seed(random_seed)
            np.random.seed(random_seed)
            tf.random.set_seed(random_seed)

    def my_metric_fn(self, y_true, y_pred):
        num = tf.reduce_mean(tf.square(y_true - y_pred))
        den = tf.reduce_mean(tf.square(y_true - tf.zeros_like(y_true)))
        return 1 - num / den

    def build(self, hp,):
        l1_reg = hp.Choice("l1_ratio", [0.,])
        l2_reg = hp.Choice("l2_ratio", [0.001,])
        dropout = hp.Choice("dropout",[0.1, 0.])
        lr = hp.Choice("learning_rate", [0.001,0.0001])
        
        # Define the hidden layers 
        input_layer = Input(shape=(self.input_value,))
        # Define hidden layers
        hidden_layer1 = Dense(64, activation ='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(input_layer)
        dropout1 = Dropout(dropout)(hidden_layer1)
        hidden_layer2 = Dense(32, activation ='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(dropout1)
        dropout2 = Dropout(dropout)(hidden_layer2)        
        hidden_layer3 = Dense(16, activation ='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(dropout2)
        dropout3 = Dropout(dropout)(hidden_layer3)
        hidden_layer4 = Dense(8, activation ='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(dropout3)
        dropout4 = Dropout(dropout)(hidden_layer4)
  
        # Concatenate the last hidden layer with the input layer 
        concatenated_layer = Concatenate()([dropout4, input_layer])
        # Define output layer 
        output_layer = Dense(1,)(concatenated_layer)
        # Create the model 
        model = Model(input_layer, output_layer)
        opt = tf.keras.optimizers.legacy.Adam(learning_rate=lr)
        model.compile(loss='mean_squared_error', optimizer=opt,metrics=[my_metric_fn])
        return model
    
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", [128,256]),
            **kwargs,)

In [8]:
def my_metric_fn(y_true, y_pred):
        num = tf.reduce_mean(tf.square(y_true - y_pred))
        den = tf.reduce_mean(tf.square(y_true))
        return 1 - num / den

def build(n_inputs):
    l1_reg = 0.0
    l2_reg = 0.001
    lr = 0.0001

    input_layer = Input(shape=(n_inputs,))
    # Define hidden layers
    hidden_layer1 = Dense(64, activation ='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(input_layer)
    #dropout1 = Dropout(dropout)(hidden_layer1)
    hidden_layer2 = Dense(32, activation ='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(hidden_layer1)
    #dropout2 = Dropout(dropout)(hidden_layer2)
    hidden_layer3 = Dense(16, activation ='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(hidden_layer2)
    #dropout3 = Dropout(dropout)(hidden_layer3)
    hidden_layer4 = Dense(8, activation ='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(hidden_layer3)
    #dropout4 = Dropout(dropout)(hidden_layer4)

    # Concatenate the last hidden layer with the input layer
    concatenated_layer = Concatenate()([hidden_layer4, input_layer])
    # Define output layer
    output_layer = Dense(1,)(concatenated_layer)
    # Create the model
    model = Model(input_layer, output_layer)
    opt = tf.keras.optimizers.legacy.Adam(learning_rate=lr)
    model.compile(loss='mean_squared_error', optimizer=opt,metrics=[my_metric_fn])
    return model

## Model 3c: only 49 Firm-level Characteristics

In [9]:
X_train, y_train, X_val, y_val, X_test, y_test = datasplit(firm_df, dataset = "firm")
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(88347, 49) (88347,) (31851, 49) (31851,) (90007, 49) (90007,)


## Nodel 3a: 49 +8

In [26]:
X_train, y_train, X_val, y_val, X_test, y_test = datasplit(merged_df, dataset = "firm_macro8")
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(88347, 57) (88347,) (31851, 57) (31851,) (90007, 57) (90007,)


In [29]:
def my_metric_fn(y_true, y_pred):
    num = tf.reduce_mean(tf.square(y_true - y_pred))
    den = tf.reduce_mean(tf.square(y_true))
    return 1 - num / den

class MyHyperModel(keras_tuner.HyperModel):
    def __init__(self,num_input):
        self.num_input = num_input

    def build(self, hp,):
        l1_reg = hp.Choice("l1_ratio", [0.])
        l2_reg = hp.Choice("l2_ratio", [0.001])
        init_mode = 'glorot_normal'
        layer = hp.Choice("layer", [4])
        acti = 'relu'
        dropout = hp.Choice("dropout",[0.1,])
        lr = hp.Choice("learning_rate", [0.0001])
        model = keras.Sequential()
        model.add(Dense(units=64, input_dim=self.num_input, kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg), kernel_initializer=init_mode, activation=acti))
        model.add(Dropout(dropout))
        model.add(BatchNormalization())
        for i in range (layer-1):
            model.add(Dense(int(32/2**i),kernel_regularizer=l1_l2(l1_reg,l2=l2_reg), kernel_initializer=init_mode, activation=acti))
            model.add(Dropout(dropout))
            model.add(BatchNormalization())
        model.add(Dense(1, kernel_initializer=init_mode)) 
        opt = tf.keras.optimizers.legacy.Adam(learning_rate=lr)
        model.compile(loss='mean_squared_error', optimizer=opt,metrics=[my_metric_fn])
        return model
    
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", [256]),
            **kwargs,)

In [ ]:
tuner = keras_tuner.RandomSearch(MyHyperModel(num_input= X_train.shape[1]),
                                 objective='val_loss',max_trials=10,overwrite=True,)
tuner.search(X_train, y_train, epochs=50, 
             validation_data=(X_val, y_val), 
             callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)])

In [31]:
# 10 Trials for Prediction
x_all = np.concatenate((X_train, X_val))
y_all = np.concatenate((y_train, y_val))
seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)
    best_model = tuner.get_best_models(num_models=1)[0]
    best_model.fit(x=x_all, y=y_all, epochs=20,verbose=0,
              callbacks=[keras.callbacks.EarlyStopping(monitor='loss',mode='min', patience=2)])
    print(best_model.evaluate(X_test,y_test))
    y_hat = best_model.predict(X_test).reshape(-1)
    print(y_hat.mean(),y_hat.std())
    yhat_df[random_seed] = y_hat
    print()

2813/2813 [==============================] - 1s 278us/step - loss: 0.0157 - my_metric_fn: -6.1807e-04
[0.015747705474495888, -0.0006180671625770628]
2813/2813 [==============================] - 1s 247us/step
0.0075678583 0.007812745

2813/2813 [==============================] - 1s 270us/step - loss: 0.0157 - my_metric_fn: -7.5035e-04
[0.01572880521416664, -0.0007503537926822901]
2813/2813 [==============================] - 1s 252us/step
0.008400231 0.0075586904

2813/2813 [==============================] - 1s 274us/step - loss: 0.0158 - my_metric_fn: -0.0086
[0.015829943120479584, -0.008584760129451752]
2813/2813 [==============================] - 1s 246us/step
0.0055350573 0.009396412

2813/2813 [==============================] - 1s 294us/step - loss: 0.0158 - my_metric_fn: -0.0015
[0.015781495720148087, -0.0014899451052770019]
2813/2813 [==============================] - 1s 248us/step
0.0062617986 0.008051113

2813/2813 [==============================] - 1s 283us/step - loss: 0.0158 

In [52]:
# 10 Trials for Prediction
x_all = np.concatenate((X_train, X_val))
y_all = np.concatenate((y_train, y_val))
seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    best_model = build(n_inputs=x_all.shape[1])
    best_model.fit(x=x_all, y=y_all, epochs=20,verbose=0,
              callbacks=[keras.callbacks.EarlyStopping(monitor='loss',mode='min', patience=2)])
    print(best_model.evaluate(X_test,y_test))
    y_hat = best_model.predict(X_test).reshape(-1)
    yhat_df[random_seed] = y_hat
    print()

2813/2813 [==============================] - 1s 260us/step - loss: 0.0159 - my_metric_fn: -0.0158
[0.015910731628537178, -0.01576908305287361]
2813/2813 [==============================] - 1s 437us/step

2813/2813 [==============================] - 1s 253us/step - loss: 0.0160 - my_metric_fn: -0.0204
[0.016005363315343857, -0.020375128835439682]
2813/2813 [==============================] - 1s 211us/step

2813/2813 [==============================] - 1s 250us/step - loss: 0.0159 - my_metric_fn: -0.0136
[0.015878602862358093, -0.013648519292473793]
2813/2813 [==============================] - 1s 225us/step

2813/2813 [==============================] - 1s 259us/step - loss: 0.0161 - my_metric_fn: -0.0285
[0.0160968117415905, -0.028465528041124344]
2813/2813 [==============================] - 1s 236us/step

2813/2813 [==============================] - 1s 250us/step - loss: 0.0162 - my_metric_fn: -0.0325
[0.016159111633896828, -0.03246806561946869]
2813/2813 [==============================] -

In [54]:
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = y_test
a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
R2 = 1-a/b
print(f"R^2 is {R2}")

path_y = 'predict/'
yhat_df.to_pickle(path_y+"model3a49_8.pkl")

R^2 is -0.017111059302194365


## Model 3b: 49 +134

In [55]:
X_train, y_train, X_val, y_val, X_test, y_test = datasplit(merged_df, dataset = "firm_macro")
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(88347, 183) (88347,) (31851, 183) (31851,) (90007, 183) (90007,)


In [ ]:
# 10 Trials for Prediction
x_all = np.concatenate((X_train, X_val))
y_all = np.concatenate((y_train, y_val))
seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    best_model = build(n_inputs=x_all.shape[1])
    best_model.fit(x=x_all, y=y_all, epochs=20,verbose=0,
              callbacks=[keras.callbacks.EarlyStopping(monitor='loss',mode='min', patience=2)])
    print(best_model.evaluate(X_test,y_test))
    y_hat = best_model.predict(X_test).reshape(-1)
    yhat_df[random_seed] = y_hat
    print()

In [58]:
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = y_test
a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
R2 = 1-a/b
print(f"R^2 is {R2}")

path_y = 'predict/'
yhat_df.to_pickle(path_y+"model3b49_134.pkl")

R^2 is -3.2964744660640193


## Finished 

In [ ]:
# 10 Trials for Prediction
x_all = np.concatenate((X_train, X_val))
y_all = np.concatenate((y_train, y_val))
best_hps = tuner.get_best_hyperparameters(5)
print(best_hps[0].values)
seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    best_model = tuner.get_best_models(3)[0]
    best_model.fit(x=x_all, y=y_all, epochs=20,
              callbacks=[keras.callbacks.EarlyStopping(monitor='loss',mode='min', patience=2)])
    print(best_model.evaluate(X_test,y_test))
    y_hat = best_model.predict(X_test).reshape(-1)
    yhat_df[random_seed] = y_hat
    print()

# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = y_test

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

In [29]:
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = y_test

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

0.004789000379233688


In [13]:
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = y_test

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
R2 = 1-a/b
print(f"R^2 is {R2}")

path_y = '\predict'
yhat_df.to_pickle(path_y+"model3c49.pkl")

R^2 is 0.005372386001615603


## Model 3a: 49 Firm-level Characteristics + 8 Macro-economic Variable 

In [5]:
X_train, y_train, X_val, y_val, X_test, y_test = datasplit(merged_df, dataset = "firm_macro8")
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(88347, 57) (88347,) (31851, 57) (31851,) (90007, 57) (90007,)


In [6]:
def build(input_value):
        l1_reg = 0.0
        l2_reg = 0.001
        lr = 0.0001
        
        # Define the hidden layers 
        input_layer = Input(shape=(input_value,))
        # Define hidden layers
        hidden_layer1 = Dense(64, activation ='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(input_layer)
        hidden_layer2 = Dense(32, activation ='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(hidden_layer1)
        hidden_layer3 = Dense(16, activation ='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(hidden_layer2)
        hidden_layer4 = Dense(8, activation ='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(hidden_layer3)
  
        # Concatenate the last hidden layer with the input layer 
        concatenated_layer = Concatenate()([hidden_layer4, input_layer])
        output_layer = Dense(1,)(concatenated_layer)
        # Create the model 
        model = Model(input_layer, output_layer)
        opt = tf.keras.optimizers.legacy.Adam(learning_rate=lr)
        model.compile(loss='mean_squared_error', optimizer=opt,metrics=[my_metric_fn])
        return model

In [ ]:
# 10 Trials for Prediction
x_all = np.concatenate((X_train, X_val))
y_all = np.concatenate((y_train, y_val))
seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    best_model = build(input_value = x_all.shape[1])
    best_model.fit(x=x_all, y=y_all, epochs=20,verbose=0,
              callbacks=[keras.callbacks.EarlyStopping(monitor='loss',mode='min', patience=2)])
    print(best_model.evaluate(X_test,y_test))
    y_hat = best_model.predict(X_test).reshape(-1)
    yhat_df[random_seed] = y_hat
    print()

# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = y_test

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

In [9]:
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = y_test

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
R2 = 1-a/b
print(f"R^2 is {R2}")

path_y = '\predict'
yhat_df.to_pickle(path_y+"model3a49_8.pkl")

R^2 is -0.017111059302194365


## Model 3b: 49 +134

In [10]:
X_train, y_train, X_val, y_val, X_test, y_test = datasplit(merged_df, dataset = "firm_macro")
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(88347, 183) (88347,) (31851, 183) (31851,) (90007, 183) (90007,)


In [ ]:
# 10 Trials for Prediction
x_all = np.concatenate((X_train, X_val))
y_all = np.concatenate((y_train, y_val))
seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    best_model = build(input_value = x_all.shape[1])
    best_model.fit(x=x_all, y=y_all, epochs=20,verbose=0,
              callbacks=[keras.callbacks.EarlyStopping(monitor='loss',mode='min', patience=2)])
    print(best_model.evaluate(X_test,y_test))
    y_hat = best_model.predict(X_test).reshape(-1)
    yhat_df[random_seed] = y_hat
    print()

In [13]:
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = y_test
a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
R2 = 1-a/b
print(f"R^2 is {R2}")

path_y = 'predict/'
yhat_df.to_pickle(path_y+"model3b49_134.pkl")

R^2 is -3.2964744660640193


## Model 3c: Only 49

In [14]:
X_train, y_train, X_val, y_val, X_test, y_test = datasplit(firm_df, dataset = "firm")
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(88347, 49) (88347,) (31851, 49) (31851,) (90007, 49) (90007,)


In [15]:
# 10 Trials for Prediction
x_all = np.concatenate((X_train, X_val))
y_all = np.concatenate((y_train, y_val))
seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    best_model = build(input_value = x_all.shape[1])
    best_model.fit(x=x_all, y=y_all, epochs=20,verbose=0,
              callbacks=[keras.callbacks.EarlyStopping(monitor='loss',mode='min', patience=2)])
    print(best_model.evaluate(X_test,y_test))
    y_hat = best_model.predict(X_test).reshape(-1)
    yhat_df[random_seed] = y_hat
    print()

2813/2813 [==============================] - 1s 240us/step - loss: 0.0157 - my_metric_fn: 0.0051
[0.01565469056367874, 0.00505579449236393]
2813/2813 [==============================] - 1s 197us/step

2813/2813 [==============================] - 1s 238us/step - loss: 0.0157 - my_metric_fn: 0.0046
[0.015681492164731026, 0.004609907511621714]
2813/2813 [==============================] - 1s 199us/step

2813/2813 [==============================] - 1s 238us/step - loss: 0.0157 - my_metric_fn: 0.0042
[0.015664024278521538, 0.0042479801923036575]
2813/2813 [==============================] - 1s 197us/step

2813/2813 [==============================] - 1s 240us/step - loss: 0.0157 - my_metric_fn: 0.0041
[0.0156777985394001, 0.004111516289412975]
2813/2813 [==============================] - 1s 227us/step

2813/2813 [==============================] - 1s 238us/step - loss: 0.0157 - my_metric_fn: 0.0039
[0.015692714601755142, 0.0038685023318976164]
2813/2813 [==============================] - 1s 198u